In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field
import os
import subprocess
import dotenv

dotenv.load_dotenv()

# 使用GPT 4模型
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

os.makedirs("./temp", exist_ok=True)

@tool(parse_docstring=True)
def write_file(filename: str, content: str) -> str:
    """写入文件到temp目录
    Args:
        filename: 文件名
        content: 文件内容
    Returns:
        操作结果
    """
    try:
        filepath = os.path.join("./temp", filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        return f"文件已成功写入: {filepath}"
    except Exception as e:
        return f"写入文件失败: {str(e)}"

@tool(parse_docstring=True)
def run_python_file(filename: str) -> str:
    """运行temp目录下的Python文件
    Args:
        filename: 文件名
    Returns:
        执行结果
    """
    try:
        filepath = os.path.join("./temp", filename)
        if not os.path.exists(filepath):
            return f"文件不存在: {filepath}"
        
        result = subprocess.run(
            ["python3", filepath], 
            capture_output=True, 
            text=True,
            cwd="./temp"
        )
        
        output = ""
        if result.stdout:
            output += f"输出:\n{result.stdout}\n"
        if result.stderr:
            output += f"错误:\n{result.stderr}\n"
        if result.returncode != 0:
            output += f"返回码: {result.returncode}\n"
            
        return output if output else "程序运行完成，无输出"
    except Exception as e:
        return f"运行Python文件失败: {str(e)}"

tools = [write_file, run_python_file]

print("React Agent环境设置完成")


In [ ]:
# 方式一：只输出工具调用（Acting Only）
def create_acting_only_agent(model, tools):
    """创建只执行行动的Agent"""
    
    system_prompt = """
    你是一个编程助手。用户会给你编程任务，你需要使用可用的工具来完成任务。
    
    重要：只调用工具，不要输出任何解释性文本。直接使用工具来完成任务。
    """
    
    llm_with_tools = model.bind_tools(tools)
    
    def agent_executor(user_input: str):
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_input)
        ]
        
        response = llm_with_tools.invoke(messages)
        return response
    
    return agent_executor

# 测试方式一
acting_agent = create_acting_only_agent(model, tools)
user_task = "请写一个Python代码来验证一维Normal分布的最大似然估计(MLE)公式，使用模拟数据进行验证"

print("=== 方式一：Acting Only ===")
print(f"用户: {user_task}")

response = acting_agent(user_task)
print(f"AI响应: {response}")

if hasattr(response, 'tool_calls') and response.tool_calls:
    print(f"AI调用工具: {[tc['name'] for tc in response.tool_calls]}")
    for tool_call in response.tool_calls:
        print(f"工具调用详情: {tool_call['name']} - {tool_call['args']}")
else:
    print("AI没有调用工具")


In [ ]:
# 方式二：先推理再行动（Reasoning + Acting）  
def create_reasoning_acting_agent(model, tools):
    """创建推理+行动的Agent"""
    
    system_prompt = """
    你是一个编程助手。用户会给你编程任务，你需要使用可用的工具来完成任务。

    重要：在每次调用工具之前，你必须先明确说明：
    1. 你的思考过程 (Thought)
    2. 你计划采取的行动 (Action) 
    3. 你期望的结果 (Expected Result)

    请按照以下格式回应：
    Thought: [你的思考过程]
    Action: [你将要执行的行动]
    Expected Result: [你期望得到的结果]
    
    然后再调用相应的工具。
    """
    
    llm_with_tools = model.bind_tools(tools)
    
    def agent_executor(user_input: str):
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_input)
        ]
        
        response = llm_with_tools.invoke(messages)
        return response
    
    return agent_executor

# 测试方式二
reasoning_agent = create_reasoning_acting_agent(model, tools)

print("\n=== 方式二：Reasoning + Acting ===")
print(f"用户: {user_task}")

response = reasoning_agent(user_task)
print(f"AI响应: {response}")

if response.content:
    print(f"AI推理: {response.content}")
    
if hasattr(response, 'tool_calls') and response.tool_calls:
    print(f"AI调用工具: {[tc['name'] for tc in response.tool_calls]}")
    for tool_call in response.tool_calls:
        print(f"工具调用详情: {tool_call['name']} - {tool_call['args']}")
else:
    print("AI没有调用工具")


In [ ]:
from langgraph.prebuilt import create_react_agent

# 方式一：使用预构建的create_react_agent
# 创建React Agent，使用相同的工具和任务
agent = create_react_agent(
    model=model,
    tools=tools,  # 使用相同的write_file和run_python_file工具
)

# 运行Agent
print("=== LangGraph方式一：create_react_agent ===")
result = agent.invoke(
    {"messages": [{"role": "user", "content": user_task}]}
)

# 打印对话历史
for message in result["messages"]:
    if hasattr(message, 'type'):
        if message.type == "human":
            print(f"用户: {message.content}")
        elif message.type == "ai":
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"AI调用工具: {[tc['name'] for tc in message.tool_calls]}")
            else:
                print(f"AI: {message.content}")
        elif message.type == "tool":
            print(f"工具结果: {message.content[:100]}...")  # 截断长输出
    else:
        print(f"消息: {message}")

print("\n--- create_react_agent的优势 ---")
print("1. 开箱即用，无需复杂配置")
print("2. 内置完整的React循环逻辑")
print("3. 自动处理工具调用和结果")
print("4. 适合快速原型开发")
print("5. 支持多轮对话和复杂推理")


In [ ]:
from typing import Annotated
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 方式二：手动构建图结构
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 创建图构建器
graph_builder = StateGraph(State)

# 定义工具
# 使用相同的工具
graph_tools = tools  # 重用之前定义的工具
llm_with_tools = model.bind_tools(graph_tools)

def chatbot(state: State):
    """聊天机器人节点"""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 添加节点
graph_builder.add_node("chatbot", chatbot)

# 添加工具节点
tool_node = ToolNode(tools=graph_tools)
graph_builder.add_node("tools", tool_node)

# 添加条件边：根据是否有工具调用来决定下一步
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# 工具执行后，返回到chatbot继续对话
graph_builder.add_edge("tools", "chatbot")

# 设置起始节点
graph_builder.add_edge(START, "chatbot")

# 编译图
graph = graph_builder.compile()

print("=== LangGraph方式二：手动构建图结构 ===")

# 简单示例执行
def demonstrate_graph(user_input: str):
    """演示图结构的执行"""
    print(f"用户输入: {user_input}")
    
    # 流式处理图的更新
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            if "messages" in value:
                last_message = value["messages"][-1]
                if hasattr(last_message, 'content') and last_message.content:
                    print(f"AI响应: {last_message.content}")

# 演示图结构
demonstrate_graph(user_task)

print("\n--- 图结构方式的优势 ---")
print("1. 完全自定义控制流")
print("2. 可以添加复杂的条件逻辑")
print("3. 易于扩展和修改")
print("4. 支持复杂的多步骤工作流")
print("5. 可视化工作流程")

print("\n--- 图结构可视化 ---")
print("可以使用以下代码查看图结构：")
print("from IPython.display import Image, display")
print("display(Image(graph.get_graph().draw_mermaid_png()))")


In [ ]:
# 清理操作
print("课程完成！")
print("本课程演示了React Agent的四种不同实现方式：")
print("1. 手动实现 - Acting Only")
print("2. 手动实现 - Reasoning + Acting")
print("3. LangGraph - create_react_agent")
print("4. LangGraph - 图结构")
print("每种方式都有其适用场景和优势。")
